In [1]:
# Import libraries
import ee
import folium
import geemap
import matplotlib.pyplot as plt


In [2]:
# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project = "class2-449500")

# Load remote sensing and geometry information
geom = ee.FeatureCollection("projects/e115-jgarciayi/assets/YanachagaChemillen").geometry().dissolve(maxError=1)
landcover = ee.ImageCollection("ESA/WorldCover/v100").first().select("Map")
forest_loss = ee.Image("UMD/hansen/global_forest_change_2023_v1_11").select("lossyear")


In [3]:
# Get center for map
center = geom.centroid().coordinates().getInfo()[::-1]

# Create folium map
m = folium.Map(location=center, zoom_start=10)

# Style parameters
landcover_vis = {
    "min": 10,
    "max": 100,
    "palette": [
        "A3CC99", "ffbb22", "ffff4c", "f096ff", "8B4513",
        "b4b4b4", "f0f0f0", "0032c8", "0096a0", "c6ffff", "000000"
    ],
    "opacity": 0.5
}
forest_vis = {"palette": ["ff0000"]}

# Convert EE images to tile layers
lc_mapid = landcover.getMapId(landcover_vis)
loss_mapid = forest_loss.selfMask().getMapId(forest_vis)

# Add to folium map
folium.TileLayer(
    tiles=lc_mapid["tile_fetcher"].url_format,
    attr="ESA WorldCover",
    name="Land Cover",
    overlay=True,
    control=True,
).add_to(m)

folium.TileLayer(
    tiles=loss_mapid["tile_fetcher"].url_format,
    attr="Hansen Forest Loss",
    name="Forest Loss",
    overlay=True,
    control=True,
).add_to(m)

# Add boundary
boundary_fc = ee.FeatureCollection([ee.Feature(geom)]).style(color="000000", fillColor="00000000", width=3)
boundary_mapid = boundary_fc.getMapId({})
folium.TileLayer(
    tiles=boundary_mapid["tile_fetcher"].url_format,
    attr="Boundary",
    name="Protected Area",
    overlay=True,
    control=True,
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Add HTML legend
legend_html = """
<div style="
  position: fixed;
  bottom: 30px;
  right: 10px;
  z-index:9999;
  background-color:white;
  padding:10px;
  border:2px solid gray;
  font-size:12px;
  box-shadow:2px 2px 6px rgba(0,0,0,0.3);
  max-height:300px;
  overflow:auto;
">
<b>Legend</b><br>
<div><span style="background:#A3CC99;width:12px;height:12px;display:inline-block;"></span> Trees</div>
<div><span style="background:#ffbb22;width:12px;height:12px;display:inline-block;"></span> Shrubland</div>
<div><span style="background:#ffff4c;width:12px;height:12px;display:inline-block;"></span> Grassland</div>
<div><span style="background:#f096ff;width:12px;height:12px;display:inline-block;"></span> Cropland</div>
<div><span style="background:#8B4513;width:12px;height:12px;display:inline-block;"></span> Built-up</div>
<div><span style="background:#b4b4b4;width:12px;height:12px;display:inline-block;"></span> Bare</div>
<div><span style="background:#0032c8;width:12px;height:12px;display:inline-block;"></span> Water</div>
<div><span style="background:#ff0000;width:12px;height:12px;display:inline-block;"></span> Forest Loss</div>
<div><span style="background:#000000;width:12px;height:12px;display:inline-block;"></span> Protected Area</div>
</div>
"""
legend = folium.Element(legend_html)
m.get_root().html.add_child(legend)

# Print the map
m

In [4]:
# Save the interactive map to HTML
m.save("bird_deforestation.html")
